Analyzing twitter data for network quality in Rwanda

Today, When people are bored at their house with this locked down one of my friend KennessaJr tweeted that he want 
to collect statistics about network usage in Rwanda, and he was lucky to collect around 200 hundred tweets where people where twittig about their internet quality and Location,
I found that it may be better to collect those tweets and run a data analysis on them and see how is the mobile internet network in Kigali.

In this book I will do an end to end Data Analysis project where I will collect data from tweets , clean them using regulars expressions and analyse them to ask the following question.
- Which provider has the fatest mobile internet in Kigali
- Which area has the best internet in Kigali
- What are mobile internet providers in Kigali?

Answer to those questions may help us to find the best ISP in Rwanda as of now.
Let start:


#### I.  Collecting Data

I will be collecting Data from a tweet posted by @KenessaJr on tweeter and start analyzing them.

In [2]:
import os
from tweepy import OAuthHandler, API, Cursor, RateLimitError, TweepError
from dotenv import load_dotenv
import logging

In [3]:
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger()

https://twitter.com/kenessajr/status/1246014150033248257
https://twitter.com/kenessajr/status/1246013771073626112

SyntaxError: invalid syntax (<ipython-input-88-d01dbe06b95a>, line 1)

In [89]:
def read_credentials():
    """
    Return users os.getenv from the environnement variable
    raise a an exception if the os.getenv are empty

    Raises:
        NotImplementedError: (description]
    """
    load_dotenv('../../.env')
    consumer_key = os.getenv('TWITTER_CONSUMER_KEY')
    consumer_secret = os.getenv('TWITTER_CONSUMER_SECRET')
    access_token = os.getenv('TWITTER_ACCESS_TOKEN')
    access_secret = os.getenv('TWITTER_ACCESS_SECRET')

    if all([consumer_key, consumer_secret, access_token, access_secret]):
        return {"consumer_key": consumer_key,
                "consumer_secret": consumer_secret,
                "access_token": access_token,
                "access_secret": access_secret}
    else:
        raise ValueError('Please add a .env file and put the os.getenv on it,\
                         refer to the sample')

In [90]:
def get_twitter_auth():
    """Setup Twitter authentication.

    Return: tweepy.OAuthHandler object
    """
    try:
        credentials = read_credentials()
        consumer_key = credentials.get('consumer_key')
        consumer_secret = credentials.get('consumer_secret')
        access_token = credentials.get('access_token')
        access_secret = credentials.get('access_secret')
    except KeyError:
        logger.error("TWITTER_*  not found\n")
        sys.exit(1)
    auth = OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_token, access_secret)
    return auth


def get_twitter_client():
    """Setup Twitter API client.

    Return: tweepy.API object
    """
    auth = get_twitter_auth()
    client = API(auth)
    try:
        client.verify_credentials()
        logger.info("Authentication OK")
    except Exception as excp:
        logging.error("Error during authentication {}".format(excp))
    return client


In [91]:
api = get_twitter_client()

NameError: name 'load_dotenv' is not defined

In [92]:
def get_replies_to_tweets(user_name, tweet_id):
    replies = Cursor(api.search, q='to:{}'.format(user_name),
                     since_id=tweet_id, tweet_mode='extended').items()
    replies_data = {'full_text': list(), 'images': list()}
    while True:
        try:
            reply = replies.next()
            if not hasattr(reply, 'in_reply_to_status_id_str'):
                continue
            if str(reply.in_reply_to_status_id) == tweet_id:
                replies_data.get('full_text').append(reply.full_text)
                if 'media'in reply.entities:
                    replies_data.get('images').append([ image.get('media_url') for image in reply.entities.get('media')])
                logger.info("reply of tweet:{}".format(reply.full_text))
        except RateLimitError as e:
            logger.error("Twitter api rate limit reached".format(e))
            time.sleep(60)
            continue

        except TweepError as e:
            logger.error("Tweepy error occured:{}".format(e))
            break
        except StopIteration:
            break
        except Exception as e:
            
            logger.exception("Failed while fetching replies {}".format(e))
            break
    return replies_data

In [93]:
replies = get_replies_to_tweets("kenessajr", "1246013771073626112")

NameError: name 'Cursor' is not defined

So far so good we have around 169 data, let collect more from another tweets 

In [94]:
replies

NameError: name 'replies' is not defined

Now we have 188 data points I think we can now start our analysis , In the next part we will be using pandas dataframe to store our data and make some analyisis on it .

In [95]:
import pandas as pd
import re
import string

In [96]:
data = pd.DataFrame.from_dict(replies,orient='index').transpose()

NameError: name 'replies' is not defined

In [97]:
data.head()

,Unnamed: 0,location,isp,speed,images,location_cleaned,network_type,isp_cleaned,speed_cleaned,speed_unit
0,0,Nyamirambo,"['Mango', 'telecom']",4.7 mbps,['http://pbs.twimg.com/media/EU1QH7iX0AAi4bN.j...,nyamirambo,NaN,mango,4.7,mbps
1,1,NaN,['@airtelrw'],NaN,['http://pbs.twimg.com/media/EU1FpbZWAAA8g6M.j...,NaN,NaN,airtel,NaN,NaN
2,2,Kimironko,[],2.5 Mbps,['http://pbs.twimg.com/media/EU1Dup5WkAAXRpk.j...,kimironko,NaN,NaN,2.5,mbps
3,3,Kanombe,[],7.9 Mbps,['http://pbs.twimg.com/media/EU0pEjWXYAQsPF_.j...,kanombe,NaN,NaN,7.9,mbps
4,4,NaN,['MTN(3G)'],NaN,['http://pbs.twimg.com/media/EU0KDBDWoAAeGYV.j...,NaN,3G,mtn,NaN,NaN


In [98]:
print(data.loc[14, :].full_text)

AttributeError: 'Series' object has no attribute 'full_text'

We can see that most of our data have **Location**: , **Speed**: and the link to the tweets as well as **ISP**

So let clean the tweet and return the information we need , this is where regex come into part

Or we can use something else, but let strech our minds with regex.

In [99]:
def get_data(row):
    """
    clean the data using regex and return specific fields
    """
    text = row.full_text
    location = re.search(r'\blocation:\s*(.*)', text, re.I)
    location = location.groups()[0] if location else None
    isp = re.search(r'\bISP:\s*(.*)', text, re.I)
    isp = isp.groups()[0].split(' ')[:-1] if isp else None
    speed = re.search(r'\bspeed:\s*(.*)', text, re.I)
    speed = speed.groups()[0] if speed else None
    return {"location" : location,
            "isp" : isp,
            "speed": speed,
            "images": row.images,
            "text": row.full_text}

In [100]:
data = data.apply(get_data, axis='columns', result_type='expand')

AttributeError: 'Series' object has no attribute 'full_text'

Let check null values 

In [101]:
data.location.isna().value_counts()

False    170
True      73
Name: location, dtype: int64

In [102]:
data.isp.isna().value_counts()

False    169
True      74
Name: isp, dtype: int64

In [103]:
data.speed.isna().value_counts()

False    122
True     121
Name: speed, dtype: int64

we can see that we have a lot of missing data , like 50 for location, 50 for isp, 87 for speed, Let me continue without them

Let me handle normal cases I will come back to the will come back to expections later.

Or will come back to cleaning later tomorow, we can call it a day

In [104]:
data.head()

,Unnamed: 0,location,isp,speed,images,location_cleaned,network_type,isp_cleaned,speed_cleaned,speed_unit
0,0,Nyamirambo,"['Mango', 'telecom']",4.7 mbps,['http://pbs.twimg.com/media/EU1QH7iX0AAi4bN.j...,nyamirambo,NaN,mango,4.7,mbps
1,1,NaN,['@airtelrw'],NaN,['http://pbs.twimg.com/media/EU1FpbZWAAA8g6M.j...,NaN,NaN,airtel,NaN,NaN
2,2,Kimironko,[],2.5 Mbps,['http://pbs.twimg.com/media/EU1Dup5WkAAXRpk.j...,kimironko,NaN,NaN,2.5,mbps
3,3,Kanombe,[],7.9 Mbps,['http://pbs.twimg.com/media/EU0pEjWXYAQsPF_.j...,kanombe,NaN,NaN,7.9,mbps
4,4,NaN,['MTN(3G)'],NaN,['http://pbs.twimg.com/media/EU0KDBDWoAAeGYV.j...,NaN,3G,mtn,NaN,NaN


In [105]:
data.to_csv('../data/speed_rwanda.csv',)

#### Data Cleaning 

In [106]:
Let go with colums by columns 

SyntaxError: invalid syntax (<ipython-input-106-c3b39fbe0f86>, line 1)

#### Location

In [107]:
data.location.isna().value_counts()

False    170
True      73
Name: location, dtype: int64

As of now the location data has 73 missing values , let us not care about the missing values and treat the uniques values in the columsn

In [108]:
len(data.location.unique())

107

In [109]:
data.location.unique()

array(['Nyamirambo ', nan, 'Kimironko', 'Kanombe', 'Muhanga', 'Sonatube',
       'Karuruma', 'kiyovu ISP: MTN https://t.co/J2uVI5muur',
       'Rubavu.                                       ISP: tnsp                                                         Network:4g https://t.co/dSTSnQ8Tg1',
       'Kicukiro ', 'Kacyiru ', 'Busanza', 'Kacyiru', 'Ruhuha, Bugesera',
       'kanombe', 'Kibagabaga', 'Remera', 'Nyamirambo', 'Rwamagana',
       'Kimironko ', 'Kicukiro', 'Lower Kiyovu ', 'kicukiro ',
       'Musanze (tetagouche) ', 'Kicukiro/ Kagarama',
       'Nyarutarama ( next to BAHO Hospital )', 'Remera -Nyabisindu',
       'Kicukiro Kagarama', 'Gisozi',
       'Rwamagana District                                                     Speed    : 5.8 Mbps                                         ISP          : 3G @MTNRwanda https://t.co/G96JriFORT',
       'Gikondo', 'kabeza kumucyo', 'Kicukiro Ngoma,  ', 'Rwinkwavu',
       'Gikondo- Nyenyeri', 'Gitega-Nyarugenge', 'Biryogo', 'Gacuriro',
   

from what we can see in the above cell people was typing their place their living in and add more additional data about the location.

We can just grap the first word they write and the do continue with it.

In [110]:
import string

In [111]:
cleaned_location = data.location.str.replace('/', ' ').str.replace('-', ' ').str.replace('|', ' ').replace(',', ' ').str.split(' ', expand=True).loc[:, 0]

In [112]:
len(cleaned_location.unique())

61

At least we are progressing we can see that data comes from 61 uniques locations

In [113]:
cleaned_location = cleaned_location.str.lower()

Once we lower the text we can see the spread has reduced.

In [114]:
len(cleaned_location.unique())

53

We are at 53 differents location in Kigali

In [115]:
cleaned_location.isna().value_counts(normalize=True)

False    0.699588
True     0.300412
Name: 0, dtype: float64

We can found that almost 30 % of data are missing but we will handle that later.

In [116]:
cleaned_location = cleaned_location.str.replace(',', '')

In [117]:
len(cleaned_location.unique())

52

In [118]:
cleaned_location.value_counts()

kicukiro           22
kimironko          16
kacyiru            14
kanombe            10
gisozi             10
gikondo             9
nyamirambo          8
remera              6
kabeza              6
musanze             4
kagarama            4
rwamagana           4
karuruma            3
rusororo            3
kibagabaga          3
kimisagara          3
rebero              2
muhanga             2
sonatubes           2
biryogo             2
gacuriro            2
kimihurura          2
muhima              2
kiyovu              2
rubilizi            2
ruyenzi             2
mumena              1
gitega              1
gihara              1
rubavu.             1
busanza             1
busogo              1
lagos.              1
lower               1
kak                 1
nyamata             1
sonatube            1
niboye              1
town                1
nyakabanda          1
ruhuha              1
busogo(musanze)     1
gasabo              1
masaka              1
gatsata             1
nyarutaram

Finaaly we can view those data in an histogram to see the spread of our location.

In [119]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
sns.set(style="white", context="talk")
rs = np.random.RandomState(8)

ModuleNotFoundError: No module named 'seaborn'

In [120]:
fig, ax  = plt.subplots(1, 1, figsize=(20, 10), sharex=False)
plt.xticks(rotation=90)
sns.barplot(x=cleaned_location.value_counts().index, y=cleaned_location.value_counts().values, palette="deep", ax=ax)
ax.axhline(0, color="k", clip_on=False)
ax.set_ylabel("Count Of Population")
ax.set_xlabel("Location")
ax.set_title('Population Location Count')
fig.savefig('../images/population_location.png')  

NameError: name 'plt' is not defined

We can conclude the analysis of the location columns here , we can come back to it later for missing values

In [121]:
data = data.assign(location_cleaned = cleaned_location)

#### TODO:
But more can be done on the locations, someone who is familiar with Rwanda Location can groups locations into sector

Let us go to the go with ISP colums

#### 2. ISP

In [122]:
data.isp.isna().value_counts(normalize=True)

False    0.695473
True     0.304527
Name: isp, dtype: float64

Again 30% of the data are missing.

In [123]:
data.isp[:10]

0     ['Mango', 'telecom']
1            ['@airtelrw']
2                       []
3                       []
4              ['MTN(3G)']
5                ['Mango']
6    ['Axiom', 'Networks']
7                      NaN
8                      NaN
9                  ['MTN']
Name: isp, dtype: object

In [124]:
We can se that  the data where splited.

SyntaxError: invalid syntax (<ipython-input-124-5d605985186f>, line 1)

In [125]:
isp = data.isp.apply(lambda x : " ".join(x) if x else None)

TypeError: can only join an iterable

In [126]:
isp.unique()

NameError: name 'isp' is not defined

From what we can see, we can:
- remove all the @ before the provider,
- remove 3G and 4G mention and save them
- Some providers put others names afters and we keep only the first words like Mango, Airtle, MTN, fatsnet

In [127]:
network_provider_regex = re.compile(r'\b3G\b | \b4G\b ', flags=re.I | re.X)

In [128]:
Removing the @ before provider.

SyntaxError: invalid syntax (<ipython-input-128-389500abe544>, line 1)

In [129]:
isp = isp.str.replace("@", '')

NameError: name 'isp' is not defined

In [130]:
network_type = isp.str.findall(network_provider_regex)

NameError: name 'isp' is not defined

In [131]:
network_type = network_type.apply(lambda x : x[0] if x else None)

NameError: name 'network_type' is not defined

In [132]:
network_type.isna().value_counts(normalize=True)

NameError: name 'network_type' is not defined

Only 20% of people gave their isp but we can keep them like that.

In [133]:
data = data.assign(network_type=network_type)

NameError: name 'network_type' is not defined

In [134]:
data.tail()

,Unnamed: 0,location,isp,speed,images,location_cleaned,network_type,isp_cleaned,speed_cleaned,speed_unit
238,238,Mumena,"['Mango', 'Telecom']",8.5 Mbps,NaN,mumena,NaN,mango,8.5,mbps
239,239,Gikondo,['MTN'],7.2 Mbps,NaN,gikondo,NaN,mtn,7.2,mbps
240,240,Kimisagara,['Airtel'],1.2 Mbps,NaN,kimisagara,NaN,airtel,1.2,mbps
241,241,Gisozi,"['CB', 'Net']",34 Mbps,NaN,gisozi,NaN,CbNet,34,mbps
242,242,Kimironko,[],4.6 Mbps,NaN,kimironko,NaN,NaN,4.6,mbps


let remove those mentions from the data

In [135]:
isp = isp.str.replace(network_provider_regex, '').str.replace('/', '').str.replace('(', '').str.replace(')', '').str.replace('-', '')

NameError: name 'isp' is not defined

In [136]:
isp.unique()

NameError: name 'isp' is not defined

In [137]:
isp = isp.str.lower().str.strip().str.split(' ', expand=True)[0]

NameError: name 'isp' is not defined

let remove 'rw or rwanda' suffix to the data and see whtat  we can have

In [138]:
isp = isp.str.replace('rwanda', '').str.replace('rw', '').str.replace('telecom', '').str.replace('isp_', '')

NameError: name 'isp' is not defined

In [139]:
isp.value_counts()

NameError: name 'isp' is not defined

We can see that most of the provideers are now presents , we can now remove word that was not well typed.

In [140]:
isp = isp.replace(to_replace={'truconnect':'trueconnect', 'trueconnect':'trueconnect', 'trukonekt':'trueconnect'})\
    .replace(to_replace='aitell', value='airtel')\
    .replace(to_replace='mangoktrn', value='mango')\
    .replace(to_replace='cb', value='CbNet')

NameError: name 'isp' is not defined

In [141]:
isp.value_counts()

NameError: name 'isp' is not defined

In [142]:
data = data.assign(isp_cleaned = isp)

NameError: name 'isp' is not defined

In [143]:
fig, ax  = plt.subplots(1, 1, figsize=(20, 10), sharex=False)
sns.barplot(x=isp.value_counts().index, y=isp.value_counts().values, palette="deep", ax=ax)
ax.axhline(0, color="k", clip_on=False)
ax.set_ylabel("Count Of Population")
ax.set_xlabel("Provider")
ax.set_title('Population per Provider')
fig.savefig('../images/population_provider.png') 

NameError: name 'plt' is not defined

As well as for the place , we can have missing value here, we can clean them after.

In [144]:
data.to_csv('../data/speed_rwanda.csv',)

III. Speed

Let see what we can do with speed data the way it comes

In [145]:
data = pd.read_csv('../data/speed_rwanda.csv')

In [146]:
data.shape

(243, 11)

It's obvious that speed comes to number and speed , in the format 56 Mbps or 6 kbps

In [147]:
data.speed.isna().value_counts(normalize=True)

False    0.502058
True     0.497942
Name: speed, dtype: float64

Almost 50% of data have missing values. We need to find a way to get more a=infos about missing data.
Something from the images.

In [148]:
data.speed.str.lower().str.strip().str.replace('⬇', '')

0      4.7 mbps
1           NaN
2      2.5 mbps
3      7.9 mbps
4           NaN
         ...   
238    8.5 mbps
239    7.2 mbps
240    1.2 mbps
241     34 mbps
242    4.6 mbps
Name: speed, Length: 243, dtype: object

In [149]:
speed_regex = re.compile(r'\s*kbps\b|\s*mbps\b', flags=re.I | re.X)

In [150]:
emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags=re.UNICODE)

In [151]:
data.speed = data.speed.replace(emoji_pattern, '')

In [152]:
speed_unit = data.speed.str.lower().str.findall(speed_regex).apply(lambda x : x[0] if (isinstance(x, list) and x) else x)
speed_unit

0       mbps
1        NaN
2       mbps
3       mbps
4        NaN
       ...  
238     mbps
239     mbps
240     mbps
241     mbps
242     mbps
Name: speed, Length: 243, dtype: object

In [153]:
speed_unit.isna().value_counts(normalize=True)

False    0.502058
True     0.497942
Name: speed, dtype: float64

In [154]:
data.shape

(243, 11)

In [155]:
speed = data.speed.replace(speed_regex, '')

In [156]:
data.shape

(243, 11)

let also get only the first float number in the digit field

In [157]:
speed = speed.str.findall(r'[0-9]*\.?[0-9]*.').apply(lambda x : x[0] if (isinstance(x, list) and x) else x)

In [158]:
speed[215].replace('⬇', '')

'120'

actually we can see that that almost what what we have is cleaned .

In [159]:
data = data.assign(speed_cleaned = speed, speed_unit=speed_unit)

In [160]:
data.shape

(243, 11)

In [161]:
data = data.assign(speed_normalize=speed_normalize)

NameError: name 'speed_normalize' is not defined

In [162]:
data.speed.str.contains('kbps').fillna(False)

0      False
1      False
2      False
3      False
4      False
       ...  
238    False
239    False
240    False
241    False
242    False
Name: speed, Length: 243, dtype: bool

In [163]:
data.loc[215, 'speed_cleaned'] = speed[215].replace('⬇', '')

In [164]:
data.loc[ np.logical_and(data.speed.str.lower().str.contains('kbps').fillna(False), ~data.speed.str.lower().str.contains('mbps').fillna(False))]

,Unnamed: 0,Unnamed: 0.1,location,isp,speed,images,location_cleaned,network_type,isp_cleaned,speed_cleaned,speed_unit
22,22,22,Kibagabaga,"['Airtel', '(4G)']",600 Kbps,['http://pbs.twimg.com/media/EUwvTF4WkAIdsA9.j...,kibagabaga,4G,airtel,600,kbps
27,27,27,Kimironko,NaN,290 Kbps,['http://pbs.twimg.com/media/EUwh8WSX0AEQBOe.j...,kimironko,NaN,NaN,290,kbps
51,51,51,"Kicukiro Ngoma,",['MTN'],490kbps https://t.co/PhVvzsjPr8,['http://pbs.twimg.com/media/EUu5JyFWsAIJuvz.j...,kicukiro,NaN,mtn,490,kbps
54,54,54,Gitega-Nyarugenge,[],80 Kbps,['http://pbs.twimg.com/media/EUulnG-WkAYiH0v.j...,gitega,NaN,NaN,80,kbps
85,85,85,Kimironko,"['Airtel', '(3G)']",38Kbps,['http://pbs.twimg.com/media/EUslhD_WAAA27yp.j...,kimironko,3G,airtel,38,kbps
88,88,88,Muhanga,['Airtel'],160 Kbps,['http://pbs.twimg.com/media/EUsfXe_XQAMiwm1.j...,muhanga,NaN,airtel,160,kbps
91,91,91,bugesera mayange,"['RTN', '', '4G', '@ktRwanda']",230kbps,['http://pbs.twimg.com/media/EUsbJGNWsAIinwi.j...,bugesera,4G,rtn,230,kbps
95,95,95,Masaka,[],550 kbps,['http://pbs.twimg.com/media/EUsXTd2XQAArvHC.j...,masaka,NaN,NaN,550,kbps
110,110,110,Kicukiro Niboye,"['MTN', '😡']",14 kbps,['http://pbs.twimg.com/media/EUsE4NZWoAAtcd_.j...,kicukiro,NaN,mtn,14,kbps
111,111,111,Nyamirambo (Kuri 40),"['@airtelrw', '', '3G']",170kbps,['http://pbs.twimg.com/media/EUsD8KPWAAEEgc_.j...,nyamirambo,3G,airtel,170,kbps


In [165]:
kps_index = data.loc[ np.logical_and(data.speed.str.lower().str.contains('kbps').fillna(False), ~data.speed.str.lower().str.contains('mbps').fillna(False))].index

In [166]:
data.loc[ np.logical_and(data.speed.str.lower().str.contains('kbps').fillna(False), ~data.speed.str.lower().str.contains('mbps').fillna(False))]

,Unnamed: 0,Unnamed: 0.1,location,isp,speed,images,location_cleaned,network_type,isp_cleaned,speed_cleaned,speed_unit
22,22,22,Kibagabaga,"['Airtel', '(4G)']",600 Kbps,['http://pbs.twimg.com/media/EUwvTF4WkAIdsA9.j...,kibagabaga,4G,airtel,600,kbps
27,27,27,Kimironko,NaN,290 Kbps,['http://pbs.twimg.com/media/EUwh8WSX0AEQBOe.j...,kimironko,NaN,NaN,290,kbps
51,51,51,"Kicukiro Ngoma,",['MTN'],490kbps https://t.co/PhVvzsjPr8,['http://pbs.twimg.com/media/EUu5JyFWsAIJuvz.j...,kicukiro,NaN,mtn,490,kbps
54,54,54,Gitega-Nyarugenge,[],80 Kbps,['http://pbs.twimg.com/media/EUulnG-WkAYiH0v.j...,gitega,NaN,NaN,80,kbps
85,85,85,Kimironko,"['Airtel', '(3G)']",38Kbps,['http://pbs.twimg.com/media/EUslhD_WAAA27yp.j...,kimironko,3G,airtel,38,kbps
88,88,88,Muhanga,['Airtel'],160 Kbps,['http://pbs.twimg.com/media/EUsfXe_XQAMiwm1.j...,muhanga,NaN,airtel,160,kbps
91,91,91,bugesera mayange,"['RTN', '', '4G', '@ktRwanda']",230kbps,['http://pbs.twimg.com/media/EUsbJGNWsAIinwi.j...,bugesera,4G,rtn,230,kbps
95,95,95,Masaka,[],550 kbps,['http://pbs.twimg.com/media/EUsXTd2XQAArvHC.j...,masaka,NaN,NaN,550,kbps
110,110,110,Kicukiro Niboye,"['MTN', '😡']",14 kbps,['http://pbs.twimg.com/media/EUsE4NZWoAAtcd_.j...,kicukiro,NaN,mtn,14,kbps
111,111,111,Nyamirambo (Kuri 40),"['@airtelrw', '', '3G']",170kbps,['http://pbs.twimg.com/media/EUsD8KPWAAEEgc_.j...,nyamirambo,3G,airtel,170,kbps


In [167]:
np.logical_and(data.speed.str.lower().str.contains('kbps').fillna(False), ~data.speed.str.lower().str.contains('mbps')

SyntaxError: unexpected EOF while parsing (<ipython-input-167-2f841446a620>, line 1)

In [168]:
def convert_kps_in_mps(row):
    print(row.speed)
    if 'kbps' in str(row.speed) and 'mbps' not in str(row.speed) and row.speed:
        return float(row.speed_cleaned)/1000
    else:
        return row.speed_cleaned

In [169]:
data.head()

,Unnamed: 0,Unnamed: 0.1,location,isp,speed,images,location_cleaned,network_type,isp_cleaned,speed_cleaned,speed_unit
0,0,0,Nyamirambo,"['Mango', 'telecom']",4.7 mbps,['http://pbs.twimg.com/media/EU1QH7iX0AAi4bN.j...,nyamirambo,NaN,mango,4.7,mbps
1,1,1,NaN,['@airtelrw'],NaN,['http://pbs.twimg.com/media/EU1FpbZWAAA8g6M.j...,NaN,NaN,airtel,NaN,NaN
2,2,2,Kimironko,[],2.5 Mbps,['http://pbs.twimg.com/media/EU1Dup5WkAAXRpk.j...,kimironko,NaN,NaN,2.5,mbps
3,3,3,Kanombe,[],7.9 Mbps,['http://pbs.twimg.com/media/EU0pEjWXYAQsPF_.j...,kanombe,NaN,NaN,7.9,mbps
4,4,4,NaN,['MTN(3G)'],NaN,['http://pbs.twimg.com/media/EU0KDBDWoAAeGYV.j...,NaN,3G,mtn,NaN,NaN


In [170]:
data.apply(convert_kps_in_mps , axis='columns')

4.7 mbps
nan
2.5 Mbps
7.9 Mbps
nan
nan
nan
3 Mbps
nan
1.2 Mbps 
nan
nan
nan
nan
3.2Mbps
nan
41 Mbps
9.3 Mbps
nan
nan
nan
nan
600 Kbps
nan
160 Mbps
1.1Mbps
nan
290 Kbps
nan
4.4 Mbps
nan
nan
4.3Mbps
2.8Mbps
nan
nan
nan
nan
11Mbps
2.9 mbps
nan
12, 13 Mbps https://t.co/STGNtSyAQS
3.3Mbps
nan
4.9 Mbps
nan
3.4Mbps(3G)
8.6Mbps
2.5Mbps
nan
nan
490kbps https://t.co/PhVvzsjPr8
nan
6.5 Mbps
80 Kbps
3.0 Mbps(metered) 384 Kbps(Actual Speed)
8.2 MPs
nan
nan
3.2Mps
nan
1.2 Mbps
3.3 Mbps
nan
8.0  Mbps
nan
nan
nan
nan
7.5 mbps
nan
3.4Mbps
nan
nan
nan
nan
nan
3.3 Mbps
8.9 KPBS
5.7Mbps
3.0 Mbps 
nan
nan
1.5KBS https://t.co/9HVBjbdxEq
3.2 Mbps 
38Kbps
3.9mbps
4.2Mbps https://t.co/ru1s5FYNoL
160 Kbps
nan
nan
230kbps
2.2 Mbps
nan
nan
550 kbps
21 Mbps 
10 Mbps
nan
nan
nan
nan
nan
nan
nan
nan
12 Mbps
250 Mbps
2.8Mbps
1.3 Mbps
14 kbps
170kbps
1.9 Mbps
3.2 Mbps
nan
20mps
140Kbps
nan
1.4 Mbps
nan
nan
130kbps 
1.5 Mbps
7.5Mbps
nan
nan
nan
3.6 Mbps
nan
nan
nan
490Kps
360Kps
nan
nan
3.2 Mbps
nan
nan
2.3 Mbps 
nan
3

0      4.7
1      NaN
2      2.5
3      7.9
4      NaN
      ... 
238    8.5
239    7.2
240    1.2
241     34
242    4.6
Length: 243, dtype: object